# Mutli Line Plot

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Repeat each element 10 times
group = np.repeat(['Group 1', 'Group 2', 'Group 3'], 10)
# Create a list of 1 - 10, 3 times
x = list(range(1,11))*3
# create an array of 30 random integers between 0 and 10
y = np.random.randint(10, size=30)
# Create a dataframe of the data
df = pd.DataFrame({'group': group,'x': x,'y': y})
df.head()

,group,x,y
0,Group 1,1,5
1,Group 1,2,9
2,Group 1,3,5
3,Group 1,4,4
4,Group 1,5,8


In [3]:
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.palettes import Category10
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [4]:
p2 = figure(plot_width=600, plot_height=300)
grp_list = df.group.unique()
grp_list

array(['Group 1', 'Group 2', 'Group 3'], dtype=object)

In [5]:
for i in range(len(grp_list)):
    source = ColumnDataSource(
        data = {'x': df.loc[df.group == grp_list[i]].x,
                'group' : df.loc[df.group == grp_list[i]].group, 
                'y': df.loc[df.group == grp_list[i]].y}
    )
    
    p2.line(x = 'x', y = 'y', source = source, legend = grp_list[i], color = (Category10[3])[i])
    
    # add tool tips
    hover = HoverTool(tooltips = [
        ('group','@group'),
        ('x','@x'),
        ('y','@y')
    ])
    p2.add_tools(hover)

show(p2)


#  Bubble Chart

In [6]:
from bokeh.models import LinearColorMapper, ColorBar, BasicTicker, PrintfTickFormatter
from bokeh.palettes import Viridis256
from bokeh.transform import transform

In [7]:
# Create the data
scale = 10
d = {'T_range': ['0-50', '0-50', '0-50', '0-50',
                 '51-60', '51-60', '51-60', '51-60',
                 '61-70', '61-70', '61-70', '61-70'],
     'Subject': ['English', 'Maths', 'Chinese', 'Arts',
                 'English', 'Maths', 'Chinese', 'Arts',
                'English', 'Maths', 'Chinese', 'Arts'],
     'count': [603, 240, 188, 89,
               220, 118, 112, 43,
              123, 2342, 32, 212],
     'count_scaled': [603 / scale, 240 / scale, 188 / scale, 89 / scale,
           220 / scale, 118 / scale, 112 / scale, 43 / scale,
          123 / scale, 2342 / scale, 32 / scale, 212 / scale]}

In [8]:
# Create the data source and the figure
df = pd.DataFrame(data = d)
source = ColumnDataSource(df)

p = figure(x_range = df['T_range'].unique(), y_range = df['Subject'].unique())

# Create the colour map
colour_mapper = LinearColorMapper(palette = Viridis256, low = df['count'].min(), high = df['count'].max())
colour_bar = ColorBar(color_mapper = colour_mapper, location = (0, 0), ticker = BasicTicker())
p.add_layout(colour_bar, 'right')
p.scatter(x = 'T_range', y = 'Subject', size = 'count_scaled', legend = None, 
          fill_color = transform('count', colour_mapper), source = source)
p.add_tools(HoverTool(tooltips = [('Count', '@count')]))

show(p)

# Gapminder Dataset

In [9]:
#import the data making the year the index
data = pd.read_csv('gapminder_tidy.csv', index_col='Year', thousands=',')
data.head()

,Country,fertility,life,population,child_mortality,gdp,region
Year,,,,,,,
1964,Afghanistan,7.671,33.639,10474903.0,339.7,1182.0,South Asia
1965,Afghanistan,7.671,34.152,10697983.0,334.1,1182.0,South Asia
1966,Afghanistan,7.671,34.662,10927724.0,328.7,1168.0,South Asia
1967,Afghanistan,7.671,35.170,11163656.0,323.3,1173.0,South Asia
1968,Afghanistan,7.671,35.674,11411022.0,318.1,1187.0,South Asia


In [26]:
from bokeh.palettes import Spectral6
from bokeh.models import NumeralTickFormatter, LinearInterpolator, CategoricalColorMapper

# get figure
hover = HoverTool(tooltips='@country : @population', show_arrow = False)
fig = figure(title = 'Life Expectancy Against GDP in 2010',
             tools=[hover], x_axis_type = 'log',
             x_range = (100, 100000),y_range = (0, 100))

#Get the 2010 data only
data_2010 = data.loc[2010]

source = ColumnDataSource(dict(x = data_2010.gdp, y = data_2010.life, 
                               country = data_2010.Country, population = data_2010.population, region = data_2010.region))

size_mapper = LinearInterpolator(x = [data.population.min(), data.population.max()],
                                 y = [5, 50])


fig.circle(x = 'x', y = 'y',
           size = {
               'field' : 'population',
               'transform' : size_mapper
           },
           alpha = 0.7,
           source = source)
fig.xaxis[0].formatter = NumeralTickFormatter(format="$0,")

show(fig)

In [11]:
life_data = data[data.Country == 'Botswana'].reset_index()
source = ColumnDataSource(data = dict(
    year = list(life_data.Year),
    life = list(life_data.life)
))

fig = figure(plot_width=900, plot_height=400, title = 'Life Expectancy Growth From 1964')

fig.line(x = 'year', y = 'life', source = source)

show(fig)

In [33]:
countries = ['Botswana', 'Zimbabwe', 'Lesotho']
fig = figure(plot_width=900, plot_height=400, title = 'Population Growth From 1964')

for i, country in enumerate(countries):
    life_data = data[data.Country == country].reset_index()
    
    source = ColumnDataSource(data = dict(year = list(life_data.Year),
                                         life = list(life_data.life),
                                         country = list(life_data.Country)))
    
    fig.line(x = 'year', y = 'life', legend = countries[i], color = Category10[3][i], source = source)
    
    # add some tool tips
    hover = HoverTool(tooltips = [
        ('Country', '@country'),
        ('Year', '@year'),
        ('Life Exp', '@life')
    ])
    fig.add_tools(hover)
    fig.xaxis.axis_label = 'Year'
    fig.yaxis.axis_label = 'Life Expectancy'

show(fig)

In [54]:
# Coloured Bubble Chart
# Colour By Region
from bokeh.palettes import Spectral6
from bokeh.models import NumeralTickFormatter, LinearInterpolator, CategoricalColorMapper

data.head()


fig = figure(title = 'Life Expectancy Against GDP in 2010', x_axis_type = 'log',
             x_range = (100, 200000),y_range = (0, 100), plot_width=1000)

#Get the 2010 data only
data_2010 = data.loc[2010]

source = ColumnDataSource(dict(x = data_2010.gdp, y = data_2010.life, 
                               country = data_2010.Country, population = data_2010.population, region = data_2010.region))
hover = HoverTool(tooltips="@country")

size_mapper = LinearInterpolator(x = [data.population.min(), data.population.max()],
                                 y = [10, 75])

colour_mapper = CategoricalColorMapper(factors = list(data.region.unique()),
                                      palette = Spectral6)

fig.circle(x = 'x', y = 'y',
           size = {
               'field' : 'population',
               'transform' : size_mapper
           },
           color = {
               'field': 'region',
               'transform': colour_mapper
           },
           alpha = 0.7,
           legend = 'region',
           source = source)
fig.xaxis[0].formatter = NumeralTickFormatter(format="$0,")
fig.add_tools(hover)
fig.legend.location = 'top_center'
fig.legend.orientation = "horizontal"

show(fig)

In [36]:
data.gdp.max()

182668.0